In [ ]:
import os
import json
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T
from tqdm import tqdm
from PIL import Image
from pathlib import Path
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix,ConfusionMatrixDisplay

## Gerekli fonksiyonlar

In [ ]:
def pad_image(image_path):
  image = Image.open(image_path).convert('RGB')
  new_img = Image.new('RGB',(350,350),color=0)
  image = T.Resize(224,max_size=320)(image)
  x = (new_img.width - image.width) // 2
  y = (new_img.height - image.height) // 2
  new_img.paste(image,(x,y))
  return new_img, new_img.size

def sort_image_list(lst):
  new_lst = []
  extensions = {}
  for i in range(0,len(lst)):
    id = Path(lst[i]).stem
    extensions[id] = lst[i][-4:]
    new_lst.append(int(id))

  new_lst = sorted(new_lst)

  for i in range(0,len(lst)):
    id = str(new_lst[i])
    new_lst[i]= id + extensions[id]
  return new_lst

def display_images_labels(imgs_lst,image_label_dictionary):
  return pd.DataFrame({
      "Image" : [img for img in imgs_lst],
      "Label" : [image_label_dictionary[img] for img in imgs_lst]
  })

def create_images_list(dir,sort=False,fullPath=False):
  lst = os.listdir(dir)

  if sort:
    lst = sort_image_list(lst)

  if fullPath:
    for i,img in enumerate(lst):
      path = os.path.join(dir,img)
      lst[i] = path

  return lst

def merge_image_directories(dir1,dir2,sort=False,fullPath=False):
  list1 = create_images_list(dir1,sort,fullPath)
  list2 = create_images_list(dir2,sort,fullPath)
  total = list1 + list2
  return total

def get_selected_indexes(indexes:list,elements:list):
  arr = []
  for idx in indexes:
    arr.append(elements[idx])
  return arr

def get_labels(images,image_lbl_dict):
  arr = []
  for img in images:
    arr.append(image_lbl_dict[img])
  return arr

## Image list must be sorted before call this function.
## For sorting call sort_image_list function before.
def create_img_label_dict(imgs_lst,labels_lst,column_num:int):
  _dict = {}
  for i,img in enumerate(imgs_lst):
    _dict[img] = labels_lst.iloc[i,column_num]
  return _dict

def create_cm(actual_labels,predicted,classes):
  cm = confusion_matrix(actual_labels, predicted)
  cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
  cm_display.plot()
  plt.title('Confusion Matrix')
  plt.show()

def search(arr,idx,element):
  if element ==-1:
    j=idx+1
    element = arr[idx]
  else:
    j=0
  while j<len(arr):
    if element == arr[j]:
      return element
    else:
      j+=1
  return -1

def getNotFoundElements(arr1,arr2):
  not_found= []
  for elm in arr1:
    xx = search(arr2,-1,elm)
    if xx ==-1:
      not_found.append(elm)
  return not_found

def load_image(image,transform):
  new_img, _ = pad_image(image)
  new_img = transform(new_img)[:3].unsqueeze(0)
  return new_img

def compute_embeddings(image_paths,device,transform,transformer):
  all_embeddings = {}
  with torch.no_grad():
    for _, img in enumerate(tqdm(image_paths,desc="Processing files")):
      embeddings = transformer(load_image(img,transform).to(device))
      all_embeddings[img] = np.array(embeddings[0].cpu().numpy()).reshape(1, -1).tolist()

  #with open("all_embeddings.json", "w") as f:
      #f.write(json.dumps(all_embeddings))
  
  lst = list(all_embeddings.values())
  return lst

def calculate_metrics(labels,predictions):
  f1 = f1_score(labels,predictions)
  precision = precision_score(labels,predictions)
  recall = recall_score(labels,predictions)
  accuracy = accuracy_score(labels,predictions)
  return accuracy, precision, recall, f1

def test_model(test_images,img_lbl_dict,classes,model,transformer,transform):
  with torch.no_grad():
    embeddings = np.array(compute_embeddings(test_images,device,transform,transformer)).reshape(-1,384)
    predictions = model.predict(embeddings)
    labels = get_labels(test_images,img_lbl_dict)
    res = calculate_metrics(labels,predictions)
    print(f"Accuracy: {res[0]}, Precision: {res[1]}, Recall: {res[2]}, F1 score: {res[3]}")
    create_cm(labels,predictions,classes)


In [ ]:
furkan_images_dir = "./dataset/Furkan_Pan/images"
gamze_images_dir = "./dataset/Gamze_Pan/images"
furkan_excel_path = "./dataset/Furkan_Excel.xlsx"
gamze_excel_path = "./dataset/Gamze_Excel.xlsx"
total_excel_path = "./dataset/Total_Excel.xlsx"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([T.ToTensor(),T.Normalize([0.5], [0.5])])
classes = ["Hatasız","Hatalı"]

In [ ]:
labels_file = pd.read_excel(total_excel_path,header=None,keep_default_na=False)
total_images_paths = merge_image_directories(furkan_images_dir,gamze_images_dir,sort=True,fullPath=True)
img_lbl_dict = create_img_label_dict(total_images_paths,labels_file,column_num=1)
pd.Series({
    "labels file length =" : len(labels_file),
    "images list length =" : len(total_images_paths),
    "dictionary  length =" : len(img_lbl_dict)
}).to_frame().style.hide(axis='columns')

In [ ]:
images_paths = random.sample(total_images_paths,1400)         ## Get 1400 sample randomly for training.
print(f"For train: {len(images_paths)}, Total: {len(total_images_paths)}")

In [ ]:
#print(f"Repeated Elements: {findRepeatedElements(images_paths)}")
test_images = getNotFoundElements(total_images_paths,images_paths)
print(f"Test images: {len(test_images)}")

In [ ]:
display_images_labels(images_paths,img_lbl_dict)

## Hangi resize işlemi kullanayım diye bu iki scripti çalıştırıyorum.


In [ ]:
path = random.choice(total_images_paths)
sample = Image.open(path).convert('RGB')
sample

In [ ]:
transformed = T.Resize((350,350))(sample)
transformed

In [ ]:
img, info = pad_image(path,224)
print(info)
img

In [ ]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vits14 = dinov2_vits14.to(device)

In [ ]:
clf = svm.SVC(probability=True,gamma='scale')               ## Initialize classifier
batch_size = 20                                             ## batch size
num_epochs = 1                                              ## Number of epochs
K = 5                                                       ## Number of folds
kf = KFold(n_splits=K)                                      ## Define K-Fold method for cross validation

In [ ]:
## Train loop
metrics = []
model = None
for i, (train_index, test_index) in enumerate(kf.split(images_paths)):
  print(f"Fold {i+1}/{K}")
  ## Get select images for train/test folds.
  x_train = get_selected_indexes(train_index,images_paths)
  x_test  = get_selected_indexes(test_index,images_paths)

  ## Get corresponding labels.
  y_train = get_labels(x_train,img_lbl_dict)
  y_test = get_labels(x_test,img_lbl_dict)

  ## Compute embeddings for train/test images.
  train_embeddings = np.array(compute_embeddings(x_train,device,transform,dinov2_vits14)).reshape(-1,384)
  test_embeddings = np.array(compute_embeddings(x_test,device,transform,dinov2_vits14)).reshape(-1,384)

  model = clf.fit(x_train,y_train)
  predictions = model.predict(x_test)

  res = calculate_metrics(y_test,predictions)
  create_cm(y_test,predictions,classes)          ## Draw confusion matrix
  metrics.append(res)

In [ ]:
## Save the model
project_dir = Path("/content/drive/MyDrive/Models/")
project_dir.mkdir(parents=True, exist_ok=True)
with open('/content/drive/MyDrive/Models/dinv2_vit14_model.pth', 'wb') as model_file:
    pickle.dump(model, model_file)

## Training sonucunda modelin performansı gösterme
## Aşağıdaki iki plot, her fold'un son accuracy ve loss değerleri gösterir.

In [ ]:
avg_acc, avg_precision, avg_recall, avg_f1= 0, 0, 0, 0
acc = []
length = len(metrics)
for i in range(length):
    avg_acc += metrics[i][0]
    acc.append(metrics[i][0])
    avg_precision += metrics[i][1]
    avg_recall += metrics[i][2]
    avg_f1 += metrics[i][3]

pd.DataFrame({
    "Average Accuracy" : 100*avg_acc/length
    "Average Precision": 100*avg_precision/length
    "Average Recall" : 100*avg_recall/length
    "Average F1 Score" : 100*avg_f1/length
})

In [ ]:
plt.plot(range(1,K+1),acc,color="blue",linewidth = 2,marker='o', markerfacecolor='red', markersize=12)
plt.title("Accuracy")
plt.xlabel("Fold")
plt.ylabel("Accuracy")
plt.show()

## Sadece 97 resim üzerinde test yapar.
Bu resimler training aşamasına dahil edilmedi.

In [ ]:
test_model(test_images,img_lbl_dict,classes,model,dinov2_vits14,transform)

## Bütün resimler için test

In [ ]:
test_model(total_images_paths,img_lbl_dict,classes,model,dinov2_vits14,transform)